In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==1

In [2]:
# # SST-2 数据集上的知识蒸馏：DeBERTa-v3-base -> 6层学生模型
#
# 本 Notebook 演示了如何在 SST-2 情感分类数据集上应用知识蒸馏技术，
# 将一个大型教师模型 (DeBERTa-v3-base) 的知识迁移到一个层数减半的小型学生模型。
#
# **主要步骤：**
# 1.  **环境设置与数据准备：** 导入库，设置环境，加载并预处理 SST-2 数据集。
# 2.  **(可选) 微调教师模型：** 本脚本假设教师模型已微调完成，直接加载。
# 3.  **创建与初始化学生模型：** 基于教师模型配置，创建层数减半的学生模型，并用教师权重初始化。
# 4.  **知识蒸馏训练：** 设置并运行蒸馏训练流程，结合标准损失和蒸馏损失。
# 5.  **评估与比较：** 评估蒸馏后的学生模型性能，并与教师模型进行对比。


In [3]:
# --------------------
# 导入基础库
# --------------------
import logging
import os

import evaluate  # Hugging Face 评估库
import numpy as np
# --------------------
# 导入核心机器学习库
# --------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset  # 加载数据集
from transformers import (
    AutoModelForSequenceClassification,  # 自动加载序列分类模型
    AutoTokenizer,  # 自动加载分词器
    DebertaV2Config,  # DeBERTa v2 配置类
    DebertaV2Model,  # DeBERTa v2 基础模型
    TrainingArguments,  # 标准训练参数配置
    Trainer,  # 标准训练器
    DataCollatorWithPadding  # 动态填充数据整理器
)
# DeBERTa v2 特定内部组件，用于权重复制
from transformers.models.deberta_v2.modeling_deberta_v2 import DebertaV2Encoder

# --------------------
# 配置与环境设置
# --------------------
# 配置日志记录器，方便追踪脚本运行过程
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 检查并设置计算设备 (优先使用 GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"使用的设备: {device}")

# 创建用于保存结果和日志的目录，如果目录已存在则忽略
os.makedirs("./result", exist_ok=True)
os.makedirs("./logs", exist_ok=True)
# 注意：模型检查点目录将由后续的 TrainingArguments 自动创建


2025-04-26 07:56:43.986798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745654204.176922      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745654204.233437      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# ## 步骤 1: 加载和预处理 SST-2 数据集
#
# 此步骤包括加载数据集、分词、格式转换和准备数据整理器及评估指标。


In [5]:
# 1.1 加载 SST-2 数据集 (来自 GLUE 基准测试)
logger.info("从 GLUE 加载 SST-2 数据集...")
sst2_dataset = load_dataset("glue", "sst2")
logger.info("原始数据集结构:")
print(sst2_dataset)


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [6]:
# 1.2 加载预训练教师模型的分词器
# 学生模型将使用与教师模型相同的分词器，以确保输入表示的一致性。
teacher_model_id = 'microsoft/deberta-v3-base'
logger.info(f"加载分词器: {teacher_model_id}")
tokenizer = AutoTokenizer.from_pretrained(teacher_model_id)


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
# 1.3 定义分词函数
# 该函数将文本输入 ('sentence' 列) 转换为模型可接受的 token ID。
# 使用 padding="max_length" 和 truncation=True 来处理不同长度的句子，
# 确保所有输入序列长度一致 (这里设置为 512)。
def tokenize_function(examples):
    """对输入的 'sentence' 列进行分词、填充和截断。"""
    return tokenizer(
        examples["sentence"],
        padding="max_length", # 填充到最大长度
        truncation=True,      # 截断超长序列
        max_length=512        # 指定最大序列长度
    )


In [8]:
# 1.4 对整个数据集应用分词函数
logger.info("开始对数据集进行分词...")
# 使用 batched=True 可以显著加速分词过程
tokenized_datasets = sst2_dataset.map(tokenize_function, batched=True)

# 1.5 数据集后处理
# 移除不再需要的原始文本列和索引列，以减少内存占用并简化后续处理
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
# 将 'label' 列重命名为 'labels'，这是 Hugging Face Trainer 期望的标签列名
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# 设置数据集格式为 PyTorch Tensors，以便与 PyTorch 模型兼容
tokenized_datasets.set_format("torch")
logger.info("分词和格式化完成。")
# (可选) 打印处理后的数据集信息或样本
# print("处理后的数据集信息:", tokenized_datasets)
# print("处理后的训练集第一个样本:", tokenized_datasets["train"][0])


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [9]:
# 1.6 准备数据整理器 (Data Collator)
# DataCollatorWithPadding 会在每个批次内部动态地将序列填充到该批次中最长序列的长度。
# 相比于在 tokenize_function 中全局填充到 max_length，这种方式通常更高效，
# 因为它可以减少不必要的填充，尤其是在批次内序列长度差异较大时。
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [10]:
# 1.7 准备评估指标计算函数
# 使用 Hugging Face Evaluate 库加载 GLUE SST-2 任务对应的准确率 (accuracy) 指标。
accuracy_metric = evaluate.load("glue", "sst2")

def compute_metrics(eval_pred):
    """
    计算模型预测的准确率。

    Args:
        eval_pred (tuple): Trainer 在评估时产生的元组，包含两个元素：
                           1. predictions (numpy.ndarray): 模型的原始输出 logits。
                           2. label_ids (numpy.ndarray): 真实的标签 ID。

    Returns:
        dict: 包含评估指标名称和值的字典，例如 {'accuracy': 0.9}。
    """
    predictions, labels = eval_pred
    # predictions 是模型的原始输出 (logits)，需要通过 argmax 找到概率最高的类别索引
    predictions = np.argmax(predictions, axis=1)
    # 使用加载的 accuracy_metric 计算准确率
    return accuracy_metric.compute(predictions=predictions, references=labels)


In [11]:
# 1.8 指定训练集、验证集和测试集
# 注意：GLUE SST-2 的官方测试集通常没有公开标签。
# 因此，在实践中，我们通常使用验证集 (validation split) 作为模型的最终测试集来评估性能。
tokenized_train = tokenized_datasets["train"]
# tokenized_train = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) # DEBUG: 可选，使用少量数据快速测试代码流程
tokenized_val = tokenized_datasets["validation"] # 用于训练过程中的模型评估和选择最佳检查点
tokenized_test = tokenized_datasets["validation"] # 用于最终模型性能评估

logger.info(f"训练集大小: {len(tokenized_train)}")
logger.info(f"验证集大小: {len(tokenized_val)}")
logger.info(f"测试集大小 (使用验证集): {len(tokenized_test)}")


In [12]:
# ## (可选) 步骤 2: 微调教师模型
#
# 知识蒸馏的前提是有一个性能良好的教师模型。这一步通常需要对预训练的教师模型 (如 DeBERTa-v3-base) 在目标任务 (SST-2) 上进行微调。
#
# **注意：** 这一步计算量较大。本脚本假设教师模型已经微调完毕，并保存在 `teacher_model_finetuned_path` 路径下。如果需要运行微调，可以取消注释相关代码或参考其他微调示例。


In [13]:
# 定义已微调教师模型的路径
teacher_model_finetuned_path = '/kaggle/input/deberta-v3-base-finetuned-sst2/deberta-v3-base-finetuned-sst2'
logger.info(f"假设已微调的教师模型路径: {teacher_model_finetuned_path}")


# 检查假设的路径是否存在，如果不存在则给出提示
if not os.path.exists(teacher_model_finetuned_path):
    logger.warning(f"未找到预期的微调教师模型路径: {teacher_model_finetuned_path}")
    logger.warning("请确保教师模型已微调并放置在此路径，或者修改路径指向正确位置。")
    # 在实际应用中，这里可能需要停止脚本或执行微调步骤
    # exit() # 或者 raise FileNotFoundError


In [14]:
# ## 步骤 3: 创建并初始化学生模型
#
# 基于已微调的教师模型，创建一个结构更小（层数减半）的学生模型，并使用教师模型的权重来初始化学生模型的部分层。这有助于学生模型在训练初期就具备一定的知识基础。


In [15]:
# 3.1 定义权重拷贝函数
def copy_deberta_weights(teacher, student):
    """
    递归地将 DeBERTa 教师模型的权重复制到学生模型。
    主要逻辑是：
    - 对于 DeBERTa 的 Encoder 部分，选择性地从教师的层复制到学生的层（例如，教师的第 0, 2, 4... 层复制到学生的第 0, 1, 2... 层）。
    - 对于模型的其他部分（如 Embeddings, Pooler, Classifier），如果结构兼容，则直接复制状态字典。

    Args:
        teacher (torch.nn.Module): 教师模型或其子模块。
        student (torch.nn.Module): 学生模型或其子模块。
    """
    # 检查是否是 DeBERTa 模型或其分类变体
    if isinstance(teacher, DebertaV2Model) or type(teacher).__name__.startswith('DebertaV2For'):
        # 递归遍历模型的子模块 (如 embeddings, encoder, pooler, classifier)
        for teacher_part, student_part in zip(teacher.children(), student.children()):
            copy_deberta_weights(teacher_part, student_part)
    # 特别处理 Encoder 部分 (包含 Transformer 层)
    elif isinstance(teacher, DebertaV2Encoder):
        # 获取教师和学生的 Transformer 层列表
        # DebertaV2Encoder -> DebertaEncoder -> ModuleList[DebertaLayer]
        teacher_encoding_layers = list(teacher.layer)
        student_encoding_layers = list(student.layer)
        num_student_layers = len(student_encoding_layers)

        logger.info(f"复制 Encoder 层: 教师层数 {len(teacher_encoding_layers)}, 学生层数 {num_student_layers}")
        # 从教师模型中选择层复制到学生模型 (这里采用每隔一层复制的策略)
        for i in range(num_student_layers):
            teacher_layer_index = 2 * i # 选择教师模型的第 0, 2, 4... 层
            if teacher_layer_index < len(teacher_encoding_layers):
                student_encoding_layers[i].load_state_dict(teacher_encoding_layers[teacher_layer_index].state_dict())
                # logger.debug(f"  学生层 {i} <- 教师层 {teacher_layer_index}")
            else:
                # 如果教师层索引超出范围 (例如，教师层数为奇数时)，则学生的部分高层可能无法从教师初始化
                logger.warning(f"教师层索引 {teacher_layer_index} 超出范围 (共 {len(teacher_encoding_layers)} 层)，学生层 {i} 未能从教师层初始化。")
    # 对于其他模块 (如 Embeddings, Pooler, Classifier)，尝试直接复制权重
    else:
        try:
            # logger.debug(f"尝试直接复制模块 {type(teacher).__name__} 的权重...")
            student.load_state_dict(teacher.state_dict())
        except RuntimeError as e:
            # 如果直接复制失败 (通常因为结构不匹配，例如分类头的输出维度不同，但这不应发生在此处，因为 num_labels 已同步)
            logger.warning(f"直接复制模块 {type(teacher).__name__} 的权重失败: {e}")
            logger.warning("请检查教师和学生模型对应部分的结构是否一致 (除了 Encoder 层数)。")
        except Exception as e:
            logger.error(f"复制模块 {type(teacher).__name__} 权重时发生未知错误: {e}")


In [16]:
# 3.2 定义创建学生模型的函数
def create_student(teacher_model):
    """
    根据给定的教师模型创建一个层数减半的学生模型。

    Args:
        teacher_model (transformers.PreTrainedModel): 预训练或微调好的教师模型实例。

    Returns:
        transformers.PreTrainedModel or None:
            一个配置和权重部分初始化自教师模型的学生模型实例。如果创建失败则返回 None。
    """
    # 获取教师模型的配置字典
    teacher_config_dict = teacher_model.config.to_dict()
    original_num_layers = teacher_config_dict.get("num_hidden_layers")

    if original_num_layers is None:
        logger.error("无法从教师模型配置中获取 'num_hidden_layers'。")
        return None

    # 计算学生模型的层数 (向下取整)
    student_num_layers = original_num_layers // 2
    if student_num_layers == 0 and original_num_layers > 0:
        logger.warning(f"教师模型层数 ({original_num_layers}) 过少，无法减半创建有意义的学生模型。")
        # 可以选择返回 None 或抛出错误
        return None
    elif student_num_layers == original_num_layers:
        logger.warning(f"教师模型层数 ({original_num_layers}) 无法减半，学生层数将与教师相同。")

    logger.info(f"教师模型层数: {original_num_layers}, 学生模型层数: {student_num_layers}")

    # 复制教师配置，并修改层数以创建学生配置
    student_config_dict = teacher_config_dict.copy()
    student_config_dict["num_hidden_layers"] = student_num_layers

    # 使用修改后的配置创建学生模型配置对象
    # 确保学生模型配置包含正确的标签数量 (用于分类头)，应与教师模型一致
    student_config = DebertaV2Config.from_dict(student_config_dict)
    student_config.num_labels = teacher_model.config.num_labels # 显式同步标签数量

    # 使用学生配置创建学生模型实例 (类型应与教师模型相同)
    # 例如, 如果 teacher 是 DebertaV2ForSequenceClassification, student 也应该是
    logger.info(f"使用配置创建学生模型实例 (类型: {type(teacher_model).__name__})...")
    student_model = type(teacher_model)(config=student_config)

    # 调用权重拷贝函数，将教师权重初始化到学生模型
    logger.info("开始从教师模型复制权重到学生模型...")
    copy_deberta_weights(teacher_model, student_model)
    logger.info("权重复制完成。")

    return student_model


In [17]:
# 3.3 加载微调好的教师模型 (用于初始化学生模型)
logger.info(f"从 {teacher_model_finetuned_path} 加载已微调的教师模型...")
try:
    # 明确指定 num_labels 以确保分类头正确加载 (虽然通常配置文件中已有，但显式指定更安全)
    teacher_model_distill_base = AutoModelForSequenceClassification.from_pretrained(
        teacher_model_finetuned_path,
        num_labels=2 # SST-2 是二分类任务
    )
except Exception as e:
    logger.error(f"加载教师模型失败: {e}")
    # 根据需要处理错误，例如退出脚本
    raise e # 重新抛出异常

# 3.4 创建学生模型实例
logger.info("创建并初始化学生模型...")
student_model = create_student(teacher_model_distill_base)

# 3.5 定义初始化后学生模型的保存路径
student_model_init_path = 'deberta-v3-student-init-sst2'

# 3.6 保存初始化后的学生模型及其配置和分词器
if student_model:
    logger.info(f"将初始化后的学生模型保存到 {student_model_init_path}")
    student_model.save_pretrained(student_model_init_path)
    # 将分词器也保存到学生模型目录，方便后续一起加载
    # 注意：此时学生模型使用的分词器与教师模型完全相同
    tokenizer.save_pretrained(student_model_init_path)
    logger.info("初始学生模型和分词器已保存。")
else:
    logger.error("学生模型创建失败，无法保存。后续步骤可能无法进行。")
    # 根据需要处理错误

# (可选) 打印学生模型结构，检查层数等信息是否符合预期
# if student_model:
#     print("\n学生模型结构:")
#     print(student_model)

# 3.7 清理内存
# 删除不再需要的教师模型和学生模型变量，释放 GPU 显存 (如果使用了 GPU)
logger.info("清理内存...")
del teacher_model_distill_base
if 'student_model' in locals() and student_model is not None: # 检查变量是否存在且不为 None
    del student_model
if torch.cuda.is_available():
    torch.cuda.empty_cache() # 尝试清空 PyTorch 的 CUDA 缓存


In [18]:
# ## 步骤 4: 设置并运行知识蒸馏训练
#
# 这一步是核心的知识蒸馏过程。我们将使用一个自定义的 `DistillationTrainer`，它在计算损失时会结合：
# 1.  **学生模型的标准交叉熵损失 (Hard Loss):** 基于真实标签计算。
# 2.  **蒸馏损失 (Soft Loss):** 通常使用 KL 散度损失，衡量学生模型输出的软概率分布与教师模型输出的软概率分布之间的差异。
#
# 损失函数通常形式为: `Loss = alpha * Loss_CE + (1 - alpha) * Loss_Distill`


In [19]:
# 4.1 定义蒸馏训练参数类
# 继承自标准的 TrainingArguments，并添加蒸馏特定的超参数 alpha 和 temperature。
class DistillationTrainingArguments(TrainingArguments):
    """
    扩展 TrainingArguments 以包含知识蒸馏所需的额外参数。

    Attributes:
        alpha (float): 平衡因子，用于调节标准交叉熵损失和蒸馏损失的权重。
                       取值范围 [0, 1]。 loss = alpha * student_loss + (1 - alpha) * distillation_loss。
        temperature (float): 温度系数，用于平滑教师和学生模型的 logits 输出，
                             产生更软的概率分布。必须大于 0。
    """
    def __init__(self, *args, alpha: float = 0.5, temperature: float = 2.0, **kwargs):
        """
        初始化蒸馏训练参数。

        Args:
            *args: 传递给 TrainingArguments 的位置参数。
            alpha (float, optional): 蒸馏损失和学生自身损失之间的平衡因子。Defaults to 0.5.
            temperature (float, optional): 用于平滑 logits 的温度系数。Defaults to 2.0.
            **kwargs: 传递给 TrainingArguments 的关键字参数。
        """
        super().__init__(*args, **kwargs)
        # 验证 alpha 和 temperature 的值
        if not 0.0 <= alpha <= 1.0:
            raise ValueError("alpha 必须在 [0, 1] 范围内")
        if not temperature > 0.0:
            raise ValueError("temperature 必须大于 0")

        self.alpha = alpha
        self.temperature = temperature


In [20]:
# 4.2 定义自定义蒸馏训练器
# 继承自标准的 Trainer，并重写 compute_loss 方法以实现蒸馏逻辑。
class DistillationTrainer(Trainer):
    """
    自定义 Trainer，用于执行知识蒸馏训练。

    在计算损失时，结合了学生模型的标准损失和与教师模型输出的 KL 散度损失。
    """
    def __init__(self, *args, teacher_model=None, **kwargs):
        """
        初始化蒸馏训练器。

        Args:
            *args: 传递给基类 Trainer 的位置参数。
            teacher_model (torch.nn.Module, optional): 教师模型实例。必须提供。
            **kwargs: 传递给基类 Trainer 的关键字参数。
        """
        super().__init__(*args, **kwargs)
        if teacher_model is None:
            raise ValueError("DistillationTrainer 需要一个 teacher_model 实例。")
        self.teacher = teacher_model
        # 确保教师模型和学生模型 (self.model) 在同一个设备上
        # self.model 是由 Trainer 基类根据 self.args.device 自动移动的
        self._move_model_to_device(self.teacher, self.model.device)
        # 将教师模型设置为评估模式 (eval mode)，因为它只用于前向传播获取 logits，
        # 不参与梯度计算和参数更新。
        self.teacher.eval()

    def compute_loss(self, model, inputs, return_outputs=False,**kwargs):
        """
        计算蒸馏损失。
        总损失 = alpha * student_cross_entropy + (1 - alpha) * kl_divergence_loss

        Args:
            model (torch.nn.Module): 学生模型 (由 Trainer 自动传入)。
            inputs (dict): 包含 input_ids, attention_mask, labels 等的模型输入字典。
                           Trainer 会自动将数据移动到正确的设备。
            return_outputs (bool, optional): 是否在返回损失的同时返回模型输出。Defaults to False.

        Returns:
            torch.Tensor or tuple[torch.Tensor, dict]:
                如果 return_outputs=False，返回计算得到的总损失 Tensor。
                如果 return_outputs=True，返回一个元组 (总损失 Tensor, 学生模型输出字典)。
        """
        # 从输入中获取真实标签
        labels = inputs.get("labels")

        # 1. 学生模型前向传播，获取学生 logits
        # model(**inputs) 会返回包含 logits 的输出对象 (例如 SequenceClassifierOutput)
        outputs_student = model(**inputs)
        logits_student = outputs_student.logits

        # 2. 计算学生模型的标准交叉熵损失 (Hard Loss)
        # 仅当提供了标签且 alpha > 0 时才需要计算此项
        student_loss = torch.tensor(0.0, device=model.device) # 初始化为 0 张量
        if labels is not None and self.args.alpha > 0:
            loss_fct_ce = nn.CrossEntropyLoss()
            # CrossEntropyLoss 期望 logits [batch_size, num_labels] 和 labels [batch_size]
            student_loss = loss_fct_ce(
                logits_student.view(-1, self.model.config.num_labels),
                labels.view(-1)
            )
        elif labels is None and self.args.alpha > 0:
            # 如果 alpha > 0 (需要学生损失) 但没有标签，发出警告
            logger.warning("计算学生损失需要标签 (labels)，但输入中未找到！学生损失将为 0。")

        # 3. 计算蒸馏损失 (Soft Loss)
        # 仅当 alpha < 1.0 时才需要计算此项
        distillation_loss = torch.tensor(0.0, device=model.device) # 初始化为 0 张量
        if self.args.alpha < 1.0:
            # 教师模型前向传播 (在 no_grad 上下文中进行，避免计算梯度)
            with torch.no_grad():
                outputs_teacher = self.teacher(**inputs)
            logits_teacher = outputs_teacher.logits.detach() # detach() 确保教师 logits 不带梯度

            # 使用温度 T 平滑教师和学生的 logits
            temperature = self.args.temperature
            # 计算 KL 散度损失。
            # nn.KLDivLoss 期望输入是 log-probabilities, 目标是 probabilities。
            # reduction='batchmean' 表示损失会在 batch 维度上取平均。
            loss_fct_kl = nn.KLDivLoss(reduction="batchmean")

            # 计算学生模型的 log-softmax 输出 (经过温度平滑)
            log_probs_student = F.log_softmax(logits_student / temperature, dim=-1)
            # 计算教师模型的 softmax 输出 (经过温度平滑)
            probs_teacher = F.softmax(logits_teacher / temperature, dim=-1)

            # 计算 KL 散度损失。乘以 temperature^2 是常见的缩放因子，
            # 用于抵消温度对梯度尺度的影响，使得蒸馏损失的量级与标准交叉熵损失更接近。
            distillation_loss = loss_fct_kl(log_probs_student, probs_teacher) * (temperature ** 2)

        # 4. 组合损失
        alpha = self.args.alpha
        loss = alpha * student_loss + (1.0 - alpha) * distillation_loss

        return (loss, outputs_student) if return_outputs else loss


In [21]:
# 4.3 加载模型准备进行蒸馏训练
# 加载教师模型 (已微调)，它将提供软标签指导学生学习
logger.info(f"加载教师模型用于蒸馏指导: {teacher_model_finetuned_path}")
try:
    teacher_model_for_distill = AutoModelForSequenceClassification.from_pretrained(
        teacher_model_finetuned_path, num_labels=2
    ).to(device) # 移动到指定设备
except Exception as e:
    logger.error(f"加载教师模型失败: {e}")
    raise e

# 加载学生模型 (已初始化)，它将接受训练
logger.info(f"加载初始化后的学生模型进行训练: {student_model_init_path}")
try:
    student_model_for_distill = AutoModelForSequenceClassification.from_pretrained(
        student_model_init_path, num_labels=2
    ).to(device) # 移动到指定设备
except Exception as e:
    logger.error(f"加载初始学生模型失败: {e}")
    raise e

# 4.4 加载分词器 (从学生模型目录加载即可，内容与教师的相同)
tokenizer = AutoTokenizer.from_pretrained(student_model_init_path)


In [22]:
# 4.5 配置蒸馏训练参数
# 定义训练过程中的检查点、日志和最终模型的保存路径
distill_output_dir = "./distill_checkpoints_sst2" # 训练检查点保存目录
distill_logging_dir = './distill_logs_sst2'       # TensorBoard 日志目录
final_student_model_path = "deberta-v3-student-distilled-sst2" # 最终最佳学生模型保存路径

# **调整蒸馏训练超参数 (针对 SST-2 数据集):**
# - num_train_epochs: 蒸馏可能需要比微调更多的轮数来让学生模型充分学习教师知识。可以从 3-5 轮开始尝试。
# - per_device_train_batch_size / gradient_accumulation_steps: 根据 GPU 显存大小调整，以达到合适的有效批次大小。
# - alpha: 控制学生损失和蒸馏损失的权重。较小的 alpha (如 0.1-0.3) 表示更依赖教师的软标签。
# - temperature: 控制 logits 的平滑程度。常用值在 2.0 到 5.0 之间，需要实验找到最佳值。
# - learning_rate: 通常使用与微调相似的学习率 (例如 2e-5 到 5e-5)，可能需要微调。
distillation_args = DistillationTrainingArguments(
    output_dir=distill_output_dir,
    # max_steps=50,                 # DEBUG: 可选，设置少量步数快速测试代码流程
    num_train_epochs=3,             # 训练轮数 (可调整)
    warmup_ratio=0.1,               # 学习率预热比例，有助于稳定训练初期
    per_device_train_batch_size=16, # 每个 GPU 的训练批次大小 (根据显存调整)
    per_device_eval_batch_size=32,  # 每个 GPU 的评估批次大小 (通常可以比训练大)
    gradient_accumulation_steps=1,  # 梯度累积步数 (如果显存不足以支持大 batch_size，可增大此值)
    learning_rate=3e-5,             # 学习率 (可调整)
    weight_decay=0.01,              # 权重衰减，防止过拟合
    logging_dir=distill_logging_dir,# TensorBoard 日志保存目录
    logging_steps=50,               # 每隔 50 步记录一次训练日志
    eval_strategy="epoch",          # 评估策略：每个 epoch 结束时进行评估
    save_strategy="epoch",          # 保存策略：每个 epoch 结束时保存检查点
    load_best_model_at_end=True,    # 训练结束后，自动加载在验证集上性能最好的模型检查点
    metric_for_best_model="accuracy",# 用于判断 "最佳模型" 的指标名称 (必须与 compute_metrics 返回的字典键匹配)
    greater_is_better=True,         # metric_for_best_model 的值越大越好
    fp16=torch.cuda.is_available(), # 如果有可用的 CUDA GPU，则启用 FP16 混合精度训练，可以加速并减少显存占用
    report_to="none",        # 将训练指标报告给 TensorBoard
    save_total_limit=2,             # 最多保存检查点的数量 (会保留最佳模型和最新的模型)
    # --- 蒸馏特定参数 ---
    alpha=0.5,                      # 学生交叉熵损失的权重 (0.3 表示更侧重蒸馏损失)
    temperature=4.0                 # 蒸馏温度 (可调整)
)

# 4.6 创建 DistillationTrainer 实例
distill_trainer = DistillationTrainer(
    model=student_model_for_distill,        # 要训练的学生模型
    teacher_model=teacher_model_for_distill,# 提供指导的教师模型
    args=distillation_args,                 # 蒸馏训练参数
    train_dataset=tokenized_train,          # 训练数据集
    eval_dataset=tokenized_val,             # 验证数据集 (用于模型选择)
    data_collator=data_collator,            # 数据整理器
    tokenizer=tokenizer,                    # 分词器
    compute_metrics=compute_metrics,        # 评估指标计算函数
)


/tmp/ipykernel_19/3337476097.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [23]:
# 4.7 开始蒸馏训练
logger.info("在 SST-2 数据集上开始知识蒸馏训练...")
try:
    train_result = distill_trainer.train()
    logger.info("知识蒸馏训练完成。")
    # (可选) 记录或打印训练过程中的一些指标
    # metrics = train_result.metrics
    # logger.info(f"训练指标: {metrics}")
    # distill_trainer.log_metrics("train", metrics)
    # distill_trainer.save_metrics("train", metrics)
except Exception as e:
    logger.error(f"蒸馏训练过程中发生错误: {e}")
    raise e


Epoch,Training Loss,Validation Loss,Accuracy
1,0.796900,1.314150,0.857798
2,0.397200,1.052762,0.886468
3,0.319600,1.041825,0.894495


In [24]:
# 4.8 保存最终训练好的学生模型
# 由于在 TrainingArguments 中设置了 load_best_model_at_end=True,
# Trainer 在训练结束后会自动加载在验证集上表现最好的模型检查点。
# 因此，调用 save_model() 会将这个最佳模型保存到指定的路径。
logger.info(f"训练结束，最佳模型检查点位于: {distill_trainer.state.best_model_checkpoint}")
logger.info(f"将最终的最佳学生模型保存到: {final_student_model_path}")
distill_trainer.save_model(final_student_model_path)
# 确保分词器也与最终模型一起保存，方便后续加载和使用
tokenizer.save_pretrained(final_student_model_path)
logger.info(f"最终学生模型和分词器已保存到 {final_student_model_path}")

# 4.9 清理内存
logger.info("清理蒸馏训练相关的模型和训练器...")
del teacher_model_for_distill
del student_model_for_distill
del distill_trainer # 删除训练器对象
if torch.cuda.is_available():
    torch.cuda.empty_cache() # 再次尝试清空 CUDA 缓存


In [25]:
# ## 步骤 5: 评估蒸馏后的学生模型
#
# 在测试集 (即 SST-2 验证集) 上评估最终得到的学生模型的性能，并可以选择性地与教师模型的性能进行比较。


In [26]:
# 5.1 加载最终蒸馏得到的学生模型
logger.info(f"加载最终蒸馏学生模型用于评估: {final_student_model_path}")
try:
    final_student_model = AutoModelForSequenceClassification.from_pretrained(
        final_student_model_path, num_labels=2
    ).to(device) # 加载到评估设备
    final_student_model.eval() # 设置为评估模式
except Exception as e:
    logger.error(f"加载最终学生模型失败: {e}")
    raise e

# 5.2 创建一个新的 Trainer 实例专门用于评估
# 这次不需要教师模型，也不需要训练相关的参数 (如学习率、优化器等)。
eval_output_dir = './eval_output_sst2' # 评估输出目录 (主要用于存放可能的日志或预测结果)
eval_args = TrainingArguments(
    output_dir=eval_output_dir,
    per_device_eval_batch_size=64,  # 评估时通常可以使用更大的批次大小
    do_train=False,                 # 不进行训练
    do_eval=True,                   # 只进行评估
    report_to="none",               # 不需要报告日志 (除非需要 TensorBoard 等记录评估结果)
    fp16=torch.cuda.is_available()  # 如果设备支持，使用 FP16 加速评估
)

# 创建评估器实例
eval_trainer = Trainer(
    model=final_student_model,      # 要评估的模型
    args=eval_args,                 # 评估参数
    eval_dataset=tokenized_test,    # 使用之前定义的测试集 (即 SST-2 验证集)
    data_collator=data_collator,    # 数据整理器
    tokenizer=tokenizer,            # 分词器 (虽然评估本身不直接用，但 Trainer 可能需要)
    compute_metrics=compute_metrics,# 评估指标计算函数
)


/tmp/ipykernel_19/2046998410.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(


In [27]:
# 5.3 在测试集上进行评估
logger.info("开始在测试集 (SST-2 验证集) 上评估最终蒸馏学生模型...")
try:
    evaluation_results = eval_trainer.evaluate() # evaluate() 方法会调用 compute_metrics
    logger.info("最终蒸馏学生模型在 SST-2 验证集上的评估结果:")
    # 打印评估结果字典, 例如: {'eval_loss': 0.3, 'eval_accuracy': 0.91, ...}
    print(evaluation_results)
    # (可选) 保存评估结果
    # eval_trainer.log_metrics("eval", evaluation_results)
    # eval_trainer.save_metrics("eval", evaluation_results)
except Exception as e:
    logger.error(f"评估学生模型时发生错误: {e}")
    raise e


{'eval_loss': 0.6753044724464417, 'eval_model_preparation_time': 0.0016, 'eval_accuracy': 0.8944954128440367, 'eval_runtime': 10.5921, 'eval_samples_per_second': 82.326, 'eval_steps_per_second': 1.322}


In [28]:
# 5.4 (可选) 评估教师模型以供比较
# 加载原始微调后的教师模型，在相同的测试集上评估其性能，以便了解蒸馏带来的性能损失。
logger.info(f"(可选比较) 加载微调教师模型进行评估: {teacher_model_finetuned_path}")
try:
    teacher_model_eval = AutoModelForSequenceClassification.from_pretrained(
        teacher_model_finetuned_path, num_labels=2
    ).to(device)
    teacher_model_eval.eval() # 设置为评估模式

    # 使用相同的评估参数创建教师模型的评估器
    teacher_eval_trainer = Trainer(
        model=teacher_model_eval,
        args=eval_args,                 # 复用之前的评估参数
        eval_dataset=tokenized_test,
        data_collator=data_collator,
        tokenizer=tokenizer,            # 确保使用相同的分词器
        compute_metrics=compute_metrics,
    )

    # 评估教师模型
    logger.info("(可选比较) 开始在测试集 (SST-2 验证集) 上评估教师模型...")
    teacher_evaluation_results = teacher_eval_trainer.evaluate()
    logger.info("微调教师模型在 SST-2 验证集上的评估结果:")
    print(teacher_evaluation_results)

    # 清理用于比较的教师模型和评估器
    logger.info("清理用于比较的教师模型...")
    del teacher_model_eval
    del teacher_eval_trainer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

except FileNotFoundError:
    logger.warning(f"未找到教师模型 {teacher_model_finetuned_path}，跳过教师模型评估比较。")
except Exception as e:
    logger.error(f"评估教师模型时发生错误: {e}")
    # 不中断主流程，仅跳过比较


/tmp/ipykernel_19/3137320589.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  teacher_eval_trainer = Trainer(


{'eval_loss': 0.1845431923866272, 'eval_model_preparation_time': 0.0026, 'eval_accuracy': 0.9610091743119266, 'eval_runtime': 20.9415, 'eval_samples_per_second': 41.64, 'eval_steps_per_second': 0.669}


In [29]:
# 5.5 (可选) 获取详细预测结果并保存
# 如果需要查看模型对具体样本的预测，可以取消注释并运行以下代码。

# logger.info("正在为测试集生成详细预测...")
# try:
#     # 使用评估器进行预测
#     prediction_outputs = eval_trainer.predict(tokenized_test)
#     # prediction_outputs.predictions 包含 logits，需要 argmax 获取最终预测标签
#     test_pred_labels = np.argmax(prediction_outputs.predictions, axis=-1)
#     logger.info(f"预测标签形状: {test_pred_labels.shape}")
#     print("预测样本 (前 20):", test_pred_labels[:20])

#     # (可选) 加载原始数据集以获取索引或其他信息进行匹配
#     # original_test_indices = sst2_dataset["validation"]["idx"] # 需要原始数据集仍然可用

#     # 将预测结果保存到 CSV 文件
#     output_df = pd.DataFrame({'predictions': test_pred_labels})
#     # (可选) 如果有原始索引，可以添加 'idx': original_test_indices
#     output_csv_path = os.path.join("./result", "distilled_student_predictions_sst2.csv")
#     output_df.to_csv(output_csv_path, index=False)
#     logger.info(f"预测结果已保存到 {output_csv_path}")

# except Exception as e:
#     logger.error(f"生成或保存预测结果时发生错误: {e}")


In [30]:
print("\n知识蒸馏脚本执行完毕。")


知识蒸馏脚本执行完毕。
